In [1]:
import pandas as pd
import numpy as np

def select_stocks(data):
    """
    基于尾盘量能-盘口-估值共振策略筛选股票
    参数:
        data: 包含所有股票字段的DataFrame
    返回:
        筛选出的股票列表(DataFrame)
    """
    # 复制数据避免修改原数据
    df = data.copy()
    
    # 1. 计算行业平均市盈率（用于估值对比）
    industry_pe = df.groupby('sw_l1_industry_name')['pe_ratio'].transform('mean')
    
    # 2. 定义所有筛选条件（严格对应MD文件字段）
    # 量能条件
    vol_cond = (df['volume_ratio_vs_5d_avg'] > 1.3) & \
               (df['auc_volume_ratio_vs_yesterday'] > 1.2) & \
               (df['money_ratio_vs_yesterday'] > 1.2)
    
    # 盘口条件
    order_cond = (df['order_book_volume_ratio'] > 1.2) & \
                 (df['buy_total'] > 500000)  # 买盘挂单总量超50万股
    
    # 估值条件
    val_cond = (df['pe_ratio'] < industry_pe * 1.1) & \
               (df['pb_ratio'] < 3)
    
    # 趋势条件
    trend_cond = (df['close'] > df['ma10']) & \
                 (df['macd_hist_calc'] > 0)
    
    # 风险过滤条件
    risk_cond = (df['paused'] == 0) & \
                (df['close'] < df['high_limit'])  # 非停牌且未涨停
    
    # 3. 综合所有条件
    final_cond = vol_cond & order_cond & val_cond & trend_cond & risk_cond
    
    # 4. 返回筛选结果（只保留关键字段）
    result = df[final_cond][[
        'date', 'stock_code', 'stock_name', 'sw_l1_industry_name',
        'close', 'volume_ratio_vs_5d_avg', 'order_book_volume_ratio',
        'pe_ratio', 'pb_ratio'
    ]].reset_index(drop=True)
    
    return result

# 示例使用
if __name__ == "__main__":
    # 假设以下是你的数据（实际使用时替换为真实数据加载）
    # 这里仅展示数据结构，字段与你的MD文件完全对应
    sample_data = pd.DataFrame({
        'date': ['2023-10-12']*100,
        'stock_code': [f'6000{i:02d}.SH' for i in range(100)],
        'stock_name': [f'股票{i}' for i in range(100)],
        'sw_l1_industry_name': np.random.choice(['医药生物', '电子', '计算机', '化工'], 100),
        'close': np.random.uniform(10, 50, 100),
        'volume_ratio_vs_5d_avg': np.random.uniform(0.8, 2.0, 100),
        'auc_volume_ratio_vs_yesterday': np.random.uniform(0.7, 1.8, 100),
        'money_ratio_vs_yesterday': np.random.uniform(0.6, 2.2, 100),
        'order_book_volume_ratio': np.random.uniform(0.9, 1.5, 100),
        'buy_total': np.random.uniform(100000, 1000000, 100),
        'pe_ratio': np.random.uniform(10, 40, 100),
        'pb_ratio': np.random.uniform(1, 5, 100),
        'ma10': np.random.uniform(9, 49, 100),
        'macd_hist_calc': np.random.uniform(-0.5, 1.0, 100),
        'paused': np.zeros(100, dtype=int),
        'high_limit': np.random.uniform(10.5, 55, 100)
    })
    
    # 执行选股
    selected = select_stocks(sample_data)
    
    # 输出结果
    print(f"选股日期: {sample_data['date'].iloc[0]}")
    print(f"符合条件的标的数量: {len(selected)}")
    if not selected.empty:
        print("\n筛选结果:")
        print(selected[['stock_code', 'stock_name', 'sw_l1_industry_name', 'close']])


选股日期: 2023-10-12
符合条件的标的数量: 0
